## Load required modules.

In [1]:
%matplotlib widget
# Import ncessary modules.
from sklearn.decomposition import PCA
import xlrd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import ipywidgets 
from ipywidgets import interact 
import ipywidgets as widgets
from matplotlib import pyplot
from IPython.display import display, clear_output, Javascript

## Read EXCEL data file into data frame.

In [2]:
# File path to your Excel file.
file_path = 'Geochemistry Results-AGG reduced variables.xlsx'

# Define the sheet name to read.
sheet_name = 'Litogeochemistry - clean'

df = pd.read_excel(file_path, sheet_name=sheet_name)

lithologies = [
    "Calcitic marble", "Dolomitic marble", "Di-Tr dolomitic marble",
    "Carbonatite", "Carbonatite-like", "Syenite", "Altered Syenite",
    "Syenite-like", "Impure Siliciclastic", "Pure Siliciclastic"
]

lithology_dropdown = widgets.SelectMultiple(
    options=lithologies,
    value = []
)

output = widgets.Output()

def filter_dataframe(change):
    global filtered_df
    selected = change['new']
    with output:
        clear_output(wait=True)
        filtered_df = df[df.iloc[:, 3].isin(selected)]
        display(filtered_df)

lithology_dropdown.observe(filter_dataframe, 'value')


print ("Hold command to deselect, or command A to select all")
display(lithology_dropdown)
display(output)



Hold command to deselect, or command A to select all


SelectMultiple(options=('Calcitic marble', 'Dolomitic marble', 'Di-Tr dolomitic marble', 'Carbonatite', 'Carbo…

Output()

In [4]:
proceed_button = widgets.Button(description="Proceed")

def on_proceed_button_clicked(b):
    b.description = "Proceeding..."
    b.disabled = False
    Javascript("Jupyter.notebook.execute_cells()")

proceed_button.on_click(on_proceed_button_clicked)
display(proceed_button)

filtered_df.head()

Button(description='Proceed', style=ButtonStyle())

,Sample Description,Year,Depth(meters),Lithology,Shape,Comment,PTS,Al2O3_pct,BaO_pct,CaO_pct,...,Cd_ppm,Co_ppm,Cu_ppm,Li_ppm,Mo_ppm,Ni_ppm,Pb_ppm,Zn_ppm,C_pct,S_pct
0,S10-01,2022,4.44,Di-Tr dolomitic marble,Marble Units,NaN,No,0.73,0.02,26.7,...,0.25,4.0,10,20,1.0,6.0,4,27,7.22,0.23
1,S10-02,2022,14.49,Dolomitic marble,Marble Units,NaN,No,0.68,0.08,30.3,...,0.25,2.0,9,20,1.0,5.0,6,22,9.99,0.75
2,S10-03,2022,15.74,Dolomitic marble,Marble Units,NaN,No,0.56,0.01,29.4,...,0.25,1.0,5,10,1.0,4.0,3,23,10.55,0.58
3,S10-04,2022,26.80,Calcitic marble,Marble Units,NaN,Yes,0.92,0.04,37.0,...,0.25,0.5,4,20,1.0,3.0,10,31,7.76,0.26
4,S10-05,2022,27.02,Calcitic marble,Marble Units,NaN,Yes,3.18,0.08,35.4,...,0.25,3.0,9,10,1.0,7.0,4,13,7.79,1.22


In [ ]:
filtered_df.shape

## Remove symbols "<" and "NaN".

In [ ]:
df = filtered_df.iloc[: , 7:]
df=df.replace('\<','',regex=True).astype(float)
df = df[np.isfinite(df).all(1)]
df = df.dropna()
df.head()

In [ ]:
df.shape

## Scale data and apply PCA.

In [ ]:
# Scale data before applying PCA
scaling=StandardScaler()

try:
    # Use fit and transform method 
    scaling.fit(df)
    Scaled_data=scaling.transform(df)
     
    # Set the n_components=3
    pca=PCA(n_components=3)
    pca.fit(Scaled_data)
    x=pca.transform(Scaled_data)
     
    # Check the dimensions of data after PCA
    print(x.shape)
except:
    raise ValueError ('Not enough components for PCA. Please select more!')
    

## Create a datafram for PCA componenets and scale it.

In [ ]:
# Create dataframe
pca_df = pd.DataFrame(
    data=x, 
    #columns=['PC1', 'PC2', 'PC3'])
    columns=['PC'+str(i) for i in range(1, len(pca.components_)+1)])
pca_df

In [ ]:
# Create the scaled PCA dataframe
pca_df_scaled = pca_df.copy()
  
scaler_df = pca_df[['PC1', 'PC2', 'PC3']]
scaler = 1 / (scaler_df.max() - scaler_df.min())
for index in scaler.index:
    pca_df_scaled[index] *= scaler[index]

## Display loadings.

In [ ]:
#loadings = pd.DataFrame(pca.components_.T, columns=['PC1', 'PC2', 'PC3'], index=df.columns)
loadings = pd.DataFrame(pca.components_.T, columns=['PC'+str(i) for i in range(1, len(pca.components_)+1)], index=df.columns)
loadings

## Display sorted loadings.

In [ ]:
# sorted_loadings = loadings.sort_values(by='PC1', ascending=False)  # Change 'ascending' as needed
# sorted_loadings

In [ ]:
# sorted_loadings = loadings.sort_values(by='PC2', ascending=False)  # Change 'ascending' as needed
# sorted_loadings

In [ ]:
# sorted_loadings = loadings.sort_values(by='PC3', ascending=False)  # Change 'ascending' as needed
# sorted_loadings

## Plot a 3D scatter plot with arrows for PCA directions.

In [ ]:
import pandas as pd

# # File path to your Excel file.
# file_path = 'Geochemistry Results-AGG reduced variables.xlsx'

# # Define the sheet name to read.
# sheet_name = 'Litogeochemistry - clean'

# # Read only the 4th column.
# dc = pd.read_excel(file_path, sheet_name=sheet_name, usecols=[3])

dc = filtered_df['Lithology']

dc = dc.to_frame()
dc = dc.rename(columns={dc.columns[0]: 'Lithology'})

color_map = {
    'Calcitic marble': 'lime',
    'Dolomitic marble': 'blue',
    'Di-Tr dolomitic marble': 'cyan',
    'Carbonatite': 'deeppink',
    'Carbonatite-like': 'pink',
    'Syenite': 'black',
    'Altered Syenite': 'black',
    'Syenite-like': 'black',
    'Impure Siliciclastic': 'grey',
    'Pure Siliciclastic': 'lightgrey'
}

dc['Color'] = dc['Lithology'].map(color_map)
#dc[dc.columns[0]] = dc[dc.columns[0]].map(color_map)

dc=dc.replace('\<','NaN',regex=True)
dc = dc[np.isfinite(df).all(1)]
dc = dc.dropna()

print(dc)
#print (pca_df_scaled)
print("Shape of y:", dc.shape)
#print("Shape of X:", X.shape)


In [ ]:
# import pandas as pd

# # File path to your Excel file.
# file_path = 'Geochemistry Results-AGG reduced variables.xlsx'

# # Define the sheet name to read.
# sheet_name = 'Litogeochemistry - clean'

# # Read only the 3rd column.
# ds = pd.read_excel(file_path, sheet_name=sheet_name, usecols=[4])


ds = filtered_df.iloc[:, 4]
ds = ds.to_frame()
ds = ds.rename(columns={ds.columns[0]: 'Shape'})

color_map1 = {
    'Marble Units': "^",
    'Altered Intrusion': "*",
    'Siliciclastic': "o",
    'Intrusion': "s",
    'Anamolous Rock': "D",
}

ds["Shapes"] = ds["Shape"].map(color_map1)
#dc[dc.columns[0]] = dc[dc.columns[0]].map(color_map)

ds=ds.replace('\<','NaN',regex=True)
ds = ds[np.isfinite(df).all(1)]
ds = ds.dropna()

print(ds)
#print (pca_df_scaled)
print("Shape of y:", ds.shape)
#print("Shape of X:", X.shape)


In [ ]:

# Initialize the 3D graph
fig = plt.figure(figsize=(9, 9))
ax = fig.add_subplot(111, projection='3d')
  
# Define scaled features as arrays
xdata = pca_df_scaled['PC1']
ydata = pca_df_scaled['PC2']
zdata = pca_df_scaled['PC3']



# Plot 3D scatterplot of PCA

#df = pd.read_excel(file_path, sheet_name=sheet_name)
#description = df.iloc[:, 0]

#descriptions = []
dc.reset_index(drop=True, inplace=True)
ds.reset_index(drop=True, inplace=True)

shapes1 = []
for i in range(len(filtered_df)):
    #print(f"Index {i}: Color {dc['Color'][i]}, Shape {ds['Shapes'][i]}")  # Debug print
    shape1 = ax.scatter3D(xdata[i], ydata[i], zdata[i], c=dc['Color'][i], marker=ds["Shapes"][i])
    shapes1.append(shape1)
    #text = ax.text(xdata[i], ydata[i], zdata[i], description[i], size=10, zorder=1, color='k')
    #descriptions.append(text)

#descriptions
dots = ax.scatter3D(xdata, ydata, zdata, c=dc['Color'])
dots.set_visible(True)

def shape(Show_Shape):
    if Show_Shape:
        for shape1 in shapes1:
            shape1.set_visible(True)  
        dots.set_visible(False) 
    else:
        dots.set_visible(True)
        for shape1 in shapes1:
            shape1.set_visible(False)

interact(shape, Show_Shape=False)



#legend
handle1 = []
label1 = []
for lithology, color in color_map.items():
    l1 = ax.scatter([], [], c=color, label=lithology)
    handle1.append(l1)
    label1.append(lithology)

handle2 = []
label2 = []
for shape, shapes in color_map1.items():
    l2 = ax.scatter([], [], c="black", marker=shapes, label=shape)
    handle2.append(l2)
    label2.append(shape)

legend1 = pyplot.legend(handle1, label1, bbox_to_anchor=(0.03, 0.2), title='Lithology', fontsize='x-small')
legend2 = pyplot.legend(handle2, label2, bbox_to_anchor=(0.03, 0.2), title='Shapes', fontsize='x-small')
ax.add_artist(legend1)
ax.add_artist(legend2)


def legenda(Legend):
    if Legend:
        legend1.set_visible(True)  
        legend2.set_visible(False)
    else:
        legend1.set_visible(False)
        legend2.set_visible(True)
interact(legenda, Legend=[('Shapes Legend', True),('Lithology Legend', False)])



# Arrows will be displayed for the following.
show_names=df.columns.tolist()
indx=df.columns.get_indexer(show_names)
scale=2

# Define the x, y, z variables
loadings = pca.components_
xs = scale*loadings[0]
ys = scale*loadings[1]
zs = scale*loadings[2]  
# Plot title of graph
plt.title(f'3D Biplot')
  
# Plot x, y, z labels
ax.set_xlabel('PC1')#, rotation=150)
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')#, rotation=60)
#plt.savefig('3D_biplot.svg')




arrow_list = []
    
# Plot the arrows
x_arr = np.zeros(len(loadings[0]))
y_arr = z_arr = x_arr
# Plot specific arrow
#indx=df.columns.get_loc("Zn_ppm")

#show_arrows = ax.quiver(x_arr[indx], y_arr[indx], z_arr[indx], xs[indx], ys[indx], zs[indx],color='r')
for i, name in enumerate(df.columns):
    ip = df.columns.get_loc(name)
    arrow = ax.quiver(x_arr[ip], y_arr[ip], z_arr[ip], xs[ip], ys[ip], zs[ip], color='r', arrow_length_ratio = 0.1, linewidth=0.5)
    arrow_list.append(arrow)
    
def findarrow(change):
    if change: 
        for a in arrow_list:
            a.set_visible(True)
    else:
        for a in arrow_list:
            a.set_visible(False)
            
# Plot the loadings
namelist = []

for i , names in enumerate(show_names):
    ip=indx[i]
    #ax.scatter(xs[ip], ys[ip], zs[ip], s=100)
    n = ax.text(xs[ip]+0.02, ys[ip]+0.02, zs[ip]+0.02, names, fontsize='small')
    namelist.append(n)

show_name_checkbox = widgets.Checkbox(value=False, description='Show all Names')
all_arrow_checkbox = widgets.Checkbox(value=False, description='Show all Arrows')

def findname(show_name):
    if show_name:
        for n in namelist:
            n.set_visible(True)  
    else:
        for n in namelist:
            n.set_visible(False)

def findname_specific(change):
    selected_items = change['new']
    if 'all' in selected_items:
        select_multiple.value = options
        for n in namelist:
            n.set_visible(True)
        for arrow in arrow_list:
            arrow.set_visible(True)
    else:
        for i in range(len(options[1:])):  
            if options[i + 1] in selected_items:
                namelist[i].set_visible(True)
                arrow_list[i].set_visible(True)
            else:
                namelist[i].set_visible(False)
                arrow_list[i].set_visible(False)
        show_name_checkbox.value = False
        all_arrow_checkbox.value = False
    plt.draw()

interact(findname, show_name=show_name_checkbox)
interact(findarrow, change=all_arrow_checkbox)

options = ['all','Al2O3_pct','BaO_pct','CaO_pct','Fe2O3_pct','K2O_pct',
           'MgO_pct','MnO_pct','Na2O_pct','P2O5_pct','SiO2_pct',
           'SrO_ppm','TiO2_pct','LOI_pct','Ce_ppm','Cs_ppm','Dy_ppm',
           'Er_ppm','Eu_ppm','Ga_ppm','Gd_ppm','Hf_ppm','Ho_ppm',
           'La_ppm','Lu_ppm','Nb_ppm','Nd_ppm','Pr_ppm','Rb_ppm',
           'Sm_ppm','Sn_ppm','Ta_ppm','Tb_ppm','Th_ppm','Tm_ppm',
           'U_ppm','V_ppm','Y_ppm','Yb_ppm','Zr_ppm','As_ppm',
           'Bi_ppm','Hg_ppm','In_ppm','Re_ppm','Sb_ppm','Sc_ppm',
           'Se_ppm','Tl_ppm','Ag_ppm','Cd_ppm','Co_ppm','Cu_ppm',
           'Li_ppm','Mo_ppm','Ni_ppm','Pb_ppm','Zn_ppm','C_pct','S_pct',]

select_multiple = widgets.SelectMultiple(
    options=options,
    value=options,
    description='Select multiple items:'
)
select_multiple.observe(findname_specific, 'value')
print ('Press Command to Select Multiple')

display(select_multiple)


In [ ]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
y = dc['Lithology']
y = y.values.ravel()
X = pca_df_scaled
print("Shape of y:", y.shape)
print("Shape of X:", X.shape)

clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X, y)

print(clf.predict([[0.3, -0.01, 0.1]]))

In [ ]:
plt.figure(figsize=(9,8))

def planes (options):
    plt.clf()
    xdata = pca_df_scaled[options[0]]
    ydata = pca_df_scaled[options[1]]
    
    shapes2 = []
    for i in range(len(df)):
        shape2 = plt.scatter(xdata[i], ydata[i], c=dc['Color'][i], marker=ds["Shapes"][i])
        shapes2.append(shape2)
    
    dots2 = plt.scatter(xdata, ydata, c=dc['Color'])
    dots2.set_visible(True)
    
    def shape(Show_Shape):
        if Show_Shape:
            for shape2 in shapes2:
                shape2.set_visible(True)  
            dots2.set_visible(False) 
        else:
            dots2.set_visible(True)
            for shape2 in shapes2:
                shape2.set_visible(False)
    
    interact(shape, Show_Shape=False)
    
    
    show_names=df.columns.tolist()
    indx=df.columns.get_indexer(show_names)
    
    
    loadings = pca.components_
    xs = loadings[options[2]]
    ys = loadings[options[3]]
    
    
    #Plot the arrows
    arrow_list_2d = []
    
    x_arr = np.zeros(len(loadings[0]))
    y_arr = x_arr
    
    arrow_scale=0.7
    for i, name in enumerate(df.columns):
        ip = df.columns.get_loc(name)
        arrow_2d = plt.quiver(x_arr[ip], y_arr[ip], xs[ip], ys[ip], color='r', scale = arrow_scale, width = 0.003)
        arrow_list_2d.append(arrow_2d)
        
    namelist_2d = []
    for i , names in enumerate(show_names):
        ip=indx[i]
        #ax.scatter(xs[ip], ys[ip], zs[ip], s=100)
        n = plt.text((1.7)*xs[ip], (1.7)*ys[ip], names, fontsize='small')
        namelist_2d.append(n)

    def findname_2d(show_name):
        if show_name:
            for n in namelist_2d:
                n.set_visible(True)  
        else:
            for n in namelist_2d:
                n.set_visible(False)
        
    def findarrow_2d(Visibility_arrow):
        if Visibility_arrow: 
            for a in arrow_list_2d:
                a.set_visible(True)
        else:
            for a in arrow_list_2d:
                a.set_visible(False)

    
    show_name_checkbox = widgets.Checkbox(value=False, description='Show all Names')
    all_arrow_checkbox = widgets.Checkbox(value=False, description='Show all Arrows')
    
    
    def findname_specific_2d(change):
        selected_items = change['new']
        for i in range(len(options[1:])): 
            if options[i] in selected_items:
                namelist_2d[i].set_visible(True)
                arrow_list_2d[i].set_visible(True)
            else:
                namelist_2d[i].set_visible(False)
                arrow_list_2d[i].set_visible(False)
        show_name_checkbox.value = False
        all_arrow_checkbox.value = False
    plt.draw()
    
    interact(findname_2d, show_name=show_name_checkbox)
    interact(findarrow_2d, Visibility_arrow=all_arrow_checkbox)
    
    options = ['Al2O3_pct','BaO_pct','CaO_pct','Fe2O3_pct','K2O_pct',
               'MgO_pct','MnO_pct','Na2O_pct','P2O5_pct','SiO2_pct',
               'SrO_ppm','TiO2_pct','LOI_pct','Ce_ppm','Cs_ppm','Dy_ppm',
               'Er_ppm','Eu_ppm','Ga_ppm','Gd_ppm','Hf_ppm','Ho_ppm',
               'La_ppm','Lu_ppm','Nb_ppm','Nd_ppm','Pr_ppm','Rb_ppm',
               'Sm_ppm','Sn_ppm','Ta_ppm','Tb_ppm','Th_ppm','Tm_ppm',
               'U_ppm','V_ppm','Y_ppm','Yb_ppm','Zr_ppm','As_ppm',
               'Bi_ppm','Hg_ppm','In_ppm','Re_ppm','Sb_ppm','Sc_ppm',
               'Se_ppm','Tl_ppm','Ag_ppm','Cd_ppm','Co_ppm','Cu_ppm',
               'Li_ppm','Mo_ppm','Ni_ppm','Pb_ppm','Zn_ppm','C_pct','S_pct',]
    
    select_multiple = widgets.SelectMultiple(
        options=options,
        value=options,
        description='Select multiple items:'
    )
    select_multiple.observe(findname_specific_2d, 'value')
    print ('Press Command to Select Multiple')
    
    display(select_multiple)

widgets.interact(planes, options = [('PC1 vs PC3', ['PC1', 'PC3', 0, 2]), ('PC2 vs PC3', ['PC2', 'PC3', 1, 2]), ('PC1 vs PC2', ['PC1', 'PC2', 0, 1])])

In [ ]:
# # import relevant libraries for 3d graph
# from mpl_toolkits.mplot3d import Axes3D
# fig = plt.figure(figsize=(10,10))
 
# # choose projection 3d for creating a 3d graph
# axis = fig.add_subplot(111, projection='3d')
 
# # x[:,0]is pc1,x[:,1] is pc2 while x[:,2] is pc3
# img=axis.scatter(x[:,0],x[:,1],x[:,2], c=df['Zn_ppm'])

# # Plot the loadings
# for i in range(5):
#     axis.scatter(xs[i], ys[i], zs[i], s=200)
#     #axis.text(
#     #    xs[i] + 0.1, 
#     #    ys[i] + 0.1, 
#     #    zs[i] + 0.1, 
#     #    varnames)
  
# # Plot the arrows
# x_arr = np.zeros(len(loadings[0]))
# y_arr = z_arr = x_arr
# axis.quiver(x_arr, y_arr, z_arr, xs, ys, zs)

# axis.set_xlabel("PC1")
# axis.set_ylabel("PC2")
# axis.set_zlabel("PC3")
# fig.colorbar(img)

In [ ]:
from sklearn.cluster import KMeans
from scipy.spatial import Voronoi, voronoi_plot_2d
from ipywidgets import interact, IntSlider
from sklearn.cluster import KMeans
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


# Remove all rows that have at least one NaN.
df = df.dropna()

def cluster_2d(axis1, axis2):
    global df
    if axis1 != axis2:
        # Remove "<" from the values.
        df[axis1] = df[axis1].apply(lambda x: x.replace('<', '') if isinstance(x, str) and '<' in x else x)
        df[axis2] = df[axis2].apply(lambda x: x.replace('<', '') if isinstance(x, str) and '<' in x else x)
        
        # Change to "object" to "float".
        df[axis1] = df[axis1].astype(float)
        df[axis2] = df[axis2].astype(float)
        
        
        # Set the data for clustering
        X = df[[axis1, axis2]].values
        
        #scaler = StandardScaler()
        #X_scaled = scaler.fit_transform(X)
        dd = pd.read_excel(file_path, sheet_name=sheet_name)
        
        #scaler and model
        def update_kmeans(k=3):    
            pipe = Pipeline([
            ("scale", StandardScaler()),
            ("model", KMeans(n_clusters=k, random_state = 0, n_init='auto'))])
        
            pipe.fit(X)
        
            # Add cluster labels to the DataFrame
            df['cluster'] = pipe.named_steps['model'].labels_
        
            x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
            y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
            xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                                 np.arange(y_min, y_max, 0.01))
        
            Z = pipe.named_steps['model'].predict(pipe.named_steps['scale'].transform(np.c_[xx.ravel(), yy.ravel()]))
            Z = Z.reshape(xx.shape)
        
            plt.figure(figsize=(9, 7))
            plt.contourf(xx, yy, Z, cmap=plt.cm.Pastel1, alpha=0.5)
            sns.scatterplot(data=df, x=axis1, y=axis2, hue='cluster', palette='viridis')
        
        
            df.apply(lambda row: plt.text(row[axis1], row[axis2], dd.iloc[row.name, 0], fontsize=7, ha='center'), axis=1)
        
            # Show the plot
            scaler = pipe.named_steps['scale']
            centroids = pipe.named_steps['model'].cluster_centers_
            centroids_original = scaler.inverse_transform(centroids)
        
            plt.scatter(centroids_original[:,0], centroids_original[:,1], 
                    marker="X", c="red", s=80, label="centroids")
            
            plt.legend()
            plt.title("K-Means Clustering")
            plt.show()
            
        interact(update_kmeans, k = IntSlider(min=2, max=10, step=1, description='N Clusters'))
    else:
        print ('select two different elements')

interact(cluster_2d, 
         axis1=widgets.Dropdown(options=df.columns.tolist(), value=df.columns[32], description='Axis1'), 
         axis2=widgets.Dropdown(options=df.columns.tolist(), value=df.columns[34], description='Axis2'))



In [ ]:
from sklearn.cluster import KMeans
from scipy.spatial import Voronoi, voronoi_plot_2d
from ipywidgets import interact, IntSlider
from sklearn.cluster import KMeans
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Remove all rows that have at least one NaN.
df = df.dropna()
dd = pd.read_excel(file_path, sheet_name=sheet_name)

def cluster_3d(axis1, axis2, axis3):
    global df
    if axis1 != axis2 != axis3:
        # Remove "<" from the values.
        for col in [axis1, axis2, axis3]:
            df[col] = df[col].apply(lambda x: x.replace('<', '') if isinstance(x, str) and '<' in x else x)
        
        # Change "object" to "float".
        df = df.astype({axis1: 'float', axis2: 'float', axis3: 'float'})
        
        # Set the data for clustering
        X = df[[axis1, axis2, axis3]].values
        
        #scaler = StandardScaler()
        #X_scaled = scaler.fit_transform(X)
        
        
        
        #scaler and model
        def update_kmeans(k=3):
            pipe = Pipeline([
                ("scale", StandardScaler()),
                ("model", KMeans(n_clusters=k, random_state=0, n_init='auto'))
            ])
            pipe.fit(X)
        
            df['cluster'] = pipe.named_steps['model'].labels_
        
            fig = plt.figure(figsize=(10, 8))
            ax = fig.add_subplot(111, projection='3d')
        
            scatter = ax.scatter(df[axis1], df[axis2], df[axis3], c=df['cluster'], cmap='viridis', s=10)
            df.apply(lambda row: ax.text(row[axis1], row[axis2], row[axis3], dd.iloc[row.name, 0], fontsize=7, ha='center'), axis=1)
        
            
        
            # Plot the centroids
            scaler = pipe.named_steps['scale']
            centroids = pipe.named_steps['model'].cluster_centers_
            centroids_original = scaler.inverse_transform(centroids)
            
            ax.scatter(centroids_original[:, 0], centroids_original[:, 1], centroids_original[:, 2], 
                       marker="X", c="red", s=100, label="centroids")
        
            ax.set_xlabel(axis1)
            ax.set_ylabel(axis2)
            ax.set_zlabel(axis3)
            plt.legend()
            plt.title("3D K-Means Clustering")
            plt.show()
        
        interact(update_kmeans, k=IntSlider(min=2, max=10, step=1, description='N Clusters'))
    else:
        print ('select three different elements')

interact(cluster_3d, 
         axis1=widgets.Dropdown(options=df.columns.tolist(), value=df.columns[0], description='Axis 1'), 
         axis2=widgets.Dropdown(options=df.columns.tolist(), value=df.columns[2], description='Axis 2'), 
         axis3=widgets.Dropdown(options=df.columns.tolist(), value=df.columns[4], description='Axis 3'))





In [ ]:
            
            # x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
            # y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
            # z_min, z_max = X[:, 2].min() - 1, X[:, 2].max() + 1
            # xx, yy, zz = np.meshgrid(np.linspace(x_min, x_max, 50),
            #                          np.linspace(y_min, y_max, 50),
            #                          np.linspace(z_min, z_max, 50))
        
            # # Predict cluster labels for each point in the mesh grid
            # grid_points = np.c_[xx.ravel(), yy.ravel(), zz.ravel()]
            # grid_points_scaled = pipe.named_steps['scale'].transform(grid_points)
            # Z = pipe.named_steps['model'].predict(grid_points_scaled)
            # Z = Z.reshape(xx.shape)
        
            # ax.scatter(xx, yy, zz, c=Z, cmap='Pastel1', alpha=0.1)
        
df.iloc[:,0]

In [ ]:
# Plot the distortion score elbow.
from yellowbrick.cluster import KElbowVisualizer

plt.figure(figsize=(10, 6))
model = KMeans(n_init='auto')
visualizer = KElbowVisualizer(model, k=(1, 12)).fit(X)
visualizer.show()